# Llama3.1 Dataset Maker for Nvidia NIM

This notebook is intended to run in a Llama3.1 NIM environment.
<br>
To set it up, please watch my video tutorial on the topic.

## Setup Llama3.1 NIM

In [2]:
%%bash 

export NGC_API_KEY= # paste your NGC API KEY here

# Log in to NGC
echo "${NGC_API_KEY}" | docker login nvcr.io -u '$oauthtoken' --password-stdin

# Set path to your LoRA model store
export LOCAL_PEFT_DIRECTORY="$(pwd)/loras"
mkdir -p $LOCAL_PEFT_DIRECTORY
pushd $LOCAL_PEFT_DIRECTORY
popd

chmod -R 777 $LOCAL_PEFT_DIRECTORY

# Set up NIM cache directory
mkdir -p $HOME/.nim-cache

export NIM_PEFT_SOURCE=/workspace/loras # Path to LoRA models internal to the container
export CONTAINER_NAME=meta-llama3_1-8b-instruct
export NIM_CACHE_PATH=$HOME/.nim-cache
export NIM_PEFT_REFRESH_INTERVAL=60

docker run -d --name=$CONTAINER_NAME \
    --network=container:verb-workspace \
    --runtime=nvidia \
    --gpus all \
    --shm-size=16GB \
    -e NGC_API_KEY \
    -e NIM_PEFT_SOURCE \
    -e NIM_PEFT_REFRESH_INTERVAL \
    -v $HOME/.nim-cache:/home/user/.nim-cache \
    -v /home/ubuntu/workspace:/workspace \
    -w /workspace \
    nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.0

# Check if NIM is up
echo "Checking if NIM is up..."
while true; do
    if curl -s http://localhost:8000 > /dev/null; then
        echo "NIM has been started successfully!"
        break
    else
        echo "NIM is not up yet. Checking again in 10 seconds..."
        sleep 10
    fi
done
 

WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credential-stores



Login Succeeded
~/verb-workspace/loras ~/verb-workspace
~/verb-workspace


Unable to find image 'nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.0' locally
1.1.0: Pulling from nim/meta/llama-3.1-8b-instruct
cbe3537751ce: Pulling fs layer
d67fcc6ef577: Pulling fs layer
47ee674c5713: Pulling fs layer
63daa0e64b30: Pulling fs layer
d9d9aecefab5: Pulling fs layer
b377c960b7f3: Pulling fs layer
071105f39313: Pulling fs layer
18049dd7c352: Pulling fs layer
071c1099eccd: Pulling fs layer
161ecdfb16f0: Pulling fs layer
fcfb2ec1ba22: Pulling fs layer
154e691e00a7: Pulling fs layer
9d18af386bf6: Pulling fs layer
f1d9f7beba6e: Pulling fs layer
0c951f04c367: Pulling fs layer
fb6fbd97005b: Pulling fs layer
431acb0bc035: Pulling fs layer
38697a17baff: Pulling fs layer
f9aeba7169f2: Pulling fs layer
cfc9a1f4fc10: Pulling fs layer
cfdd2bb2b4a6: Pulling fs layer
c396a58289c6: Pulling fs layer
63daa0e64b30: Waiting
e8839de7b7ae: Pulling fs layer
7941e23182d8: Pulling fs layer
d9d9aecefab5: Waiting
0372c9b9cb47: Pulling fs layer
161ecdfb16f0: Waiting
b377c960b7f3: Waiting
dfedf8154b

aeacfb0620ecfb3eec8141e58f5afbabce74e0e0e527b87e21e35935116ddb1e
Checking if NIM is up...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 seconds...
NIM is not up yet. Checking again in 10 sec

## Verify that Llama is Available
Specify the client URL to check if Llama3.1 is listening on port 8000

In [3]:
!curl localhost:8000/v1/health/ready

{"object":"health.response","message":"Service is ready."}

## Install OpenAI
To run the dataset maker, the openai modeule is required

In [5]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 33.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


## Create Dataset

In the following cell, you will see how to chain prompt output to generate datasets automatically.

In [13]:
from openai import OpenAI
import pandas as pd

# create empty dataframe 
data = pd.DataFrame(columns=["country", "capital", "food"])

# specify model location
client = OpenAI(
  base_url = "http://localhost:8000/v1",
  api_key = "not_used"
)

def ask_question(user_input):
    
    # specify model settings
    chat_response = client.chat.completions.create(
    # please note: model name written without a dot symbol in this implementation
    model="meta/llama-3_1-8b-instruct",
    messages=[{"role":"user","content": user_input}],
    temperature=0.5,
    top_p=1,
    max_tokens=1024,
    # return output as a single unit of text
    stream=False
    )

    return chat_response.choices[0].message.content

# fetch names of all world countries
all_countries = ask_question("""
names of all countries separated by commas in an alphabetical order.
names only, with no other output
""")
all_countries = all_countries.split(", ")

# iterate over all country names
for i, country in enumerate(all_countries):
    # fetch attributes for each country
    capital = ask_question("what is the capital city of " + country + ". just the name")
    food = ask_question("what is the national food of " + country + ". just the name")
    # store country and attributes in the pre-defined dataframe
    data.loc[i] = [country, capital, food]

# save CSV file in the current directory
data.to_csv("data.csv", header=None)
print("CSV file was successully saved in the root directory of your Jupyter Lab! :)")